In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import xarray as xr
import numpy as np
import pandas as pd

In [4]:
xr.set_options(keep_attrs=True)

In [7]:
import ecephys_analyses as ea 
from ecephys import utils
from ecephys.signal.xarray_utils import filter_dataset_by_hypnogram

In [8]:
def ds2df(ds):
    s = ds.to_dataframe().stack()
    s.index.names = s.index.names[:-1] + ['region']
    return s.to_frame(name='power')

In [9]:
subject = "Segundo"
experiment = "sleep-homeostasis"
condition = "recovery-sleep"
states = ["N1", "N2"]
duration = '02:00:00'
f_range = (0.5, 4)

In [13]:
hyp = ea.load_hypnogram(subject, experiment, condition)
power = ea.load_power(subject, experiment, condition, "spg.nc").sel(frequency=slice(*f_range)).sum(dim='frequency')

In [14]:
early_hyp = hyp.keep_states(states).keep_first(duration)
early_pow = filter_dataset_by_hypnogram(power, early_hyp).swap_dims({'time': 'datetime'}).drop_vars(['time', 'timedelta'])

In [15]:
late_hyp = hyp.keep_states(states).keep_last(duration)
late_pow = filter_dataset_by_hypnogram(power, late_hyp).swap_dims({'time': 'datetime'}).drop_vars(['time', 'timedelta'])

In [16]:
nrem_pow = pd.concat([ds2df(early_pow).assign(condition="early"), ds2df(late_pow).assign(condition="late")])

In [17]:
g = sns.displot(data=nrem_pow, x='power', hue='condition', col='region', kind='hist')
g.set(xlabel='Delta (0.5 - 4Hz) PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
nrem_pow_log = nrem_pow.copy()
nrem_pow_log['power'] = np.log(nrem_pow.power)

g = sns.displot(data=nrem_pow_log, x='power', hue='condition', col='region', kind='hist')
g.set(xlabel='Delta (0.5 - 4Hz) PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
nrem_pow_sqrt = nrem_pow.copy()
nrem_pow_sqrt['power'] = np.sqrt(nrem_pow.power)

g = sns.displot(data=nrem_pow_sqrt, x='power', hue='condition', col='region', kind='hist')
g.set(xlabel='Delta (0.5 - 4Hz) PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
def difference_of_means(df):
    means = df.groupby(['condition', 'region']).mean()
    return means.xs('early') - means.xs('late')

In [21]:
observed = difference_of_means(nrem_pow)

n_permutations = 10000
permutations = list()
permuted = nrem_pow.copy()
for i in range(n_permutations):
    np.random.shuffle(permuted.condition.values)
    permutations.append(difference_of_means(permuted))
    
df = pd.concat(permutations)

In [22]:
g = sns.displot(data=df, x='power', col='region', kind='hist')
g.set(xlabel='Difference of means (Early - Late)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
cx = df.xs('mpta_wm_ref').sort_values('power')
hc = df.xs('sr_wm_ref').sort_values('power')

In [24]:
pcx = np.sum(cx.power.values >= observed.xs('mpta_wm_ref').values)

In [25]:
phc = np.sum(hc.power.values >= observed.xs('sr_wm_ref').values)

In [26]:
print(f"Subject: {subject}")
print('Observed differences, Mean Early - Mean Late:\n')
print(observed.to_markdown())
print('\n')
print('Shuffled differences >= observed:')
print(f"Cortex: {1 + pcx} / {n_permutations}")
print(f"Hippocampus: {1 + phc} / {n_permutations}")

Subject: Segundo
Observed differences, Mean Early - Mean Late:

| region      |    power |
|:------------|---------:|
| mpta_wm_ref | 184050   |
| sr_wm_ref   |  82041.2 |


Shuffled differences >= observed:
Cortex: 1 / 10000
Hippocampus: 1 / 10000
